## Start by setting week number
#### Wait for ESPN data update until at least Wednesday morning, but run latest by Thursday night

https://www.espn.com/nfl/stats/team/_/stat/total/season/2022/seasontype/2

In [1]:
SET_WEEK = 11
week_name = 'week_' + str(SET_WEEK)
print(week_name)

week_11


In [2]:
from scrapers.scrape_2022 import get_2022_data
import pandas as pd
import numpy as np
import pickle

import warnings
warnings.filterwarnings('ignore')

### Models

In [3]:
logit = '../modeling/logit.pkl'
rf = '../modeling/rf.pkl'
gbm = '../modeling/gbm.pkl'
lgb = '../modeling/lgb.pkl'
catboost = '../modeling/catboost.pkl'

with open(logit, 'rb') as file:
    LOGIT = pickle.load(file)

with open(rf, 'rb') as file:
    RF = pickle.load(file)
    
with open(gbm, 'rb') as file:
    GBM = pickle.load(file)
    
with open(lgb, 'rb') as file:
    LGB = pickle.load(file)
    
with open(catboost, 'rb') as file:
    CATBOOST= pickle.load(file)

### 2022 data

In [4]:
%%time

X = get_2022_data()

# from boruta elimination method:
keep_cols = ['offense_points_per_game',
             'offense_downs_Fourth Downs_PCT',
             'offense_passing_AVG',
             'offense_passing_YDS/G',
             'offense_passing_RTG',
             'offense_receiving_AVG',
             'defense_points_per_game',
             'defense_passing_AVG',
             'defense_receiving_AVG',
             'defense_rushing_YDS/G',
             'defense_passing_SYL_per_game',
             'offense_downs_Third Downs_ATT_per_game',
             'defense_downs_Third Downs_ATT_per_game',
             'defense_downs_First Downs_penalty_ratio',
             'offense_passing_TD_per_game',
             'defense_rushing_TD_per_game',
             'defense_pass_TD_per_rush_TD',
             'offense_pass_TD_to_INT']

X = X[['Team', 'games_played', 'season'] + keep_cols].copy()

X.head(2)

Wall time: 28.4 s


,Team,games_played,season,offense_points_per_game,offense_downs_Fourth Downs_PCT,offense_passing_AVG,offense_passing_YDS/G,offense_passing_RTG,offense_receiving_AVG,defense_points_per_game,...,defense_receiving_AVG,defense_rushing_YDS/G,defense_passing_SYL_per_game,offense_downs_Third Downs_ATT_per_game,defense_downs_Third Downs_ATT_per_game,defense_downs_First Downs_penalty_ratio,offense_passing_TD_per_game,defense_rushing_TD_per_game,defense_pass_TD_per_rush_TD,offense_pass_TD_to_INT
0,Kansas City Chiefs,10,2022,30.0,75.0,8.1,314.7,106.6,12.3,23.3,...,10.6,100.6,21.8,11.800000,12.900000,0.054455,2.800000,0.600000,3.50,4.000000
1,Buffalo Bills,11,2022,28.1,50.0,7.7,279.2,94.1,12.1,18.1,...,10.5,105.6,18.0,11.636364,12.727273,0.096070,2.090909,0.727273,1.75,2.090909


In [5]:
print(X.shape)

(32, 21)


In [6]:
X.fillna(0, inplace = True)
X.replace([np.inf, -np.inf], 0, inplace=True)

In [7]:
X['games_played'].unique().tolist()

[10, 11]

In [8]:
# X[X['games_played'] == 8]

In [9]:
predictions = X[['Team', 'season']].copy()
predictions['week_marker'] = week_name

predictions['pred_logit'] = LOGIT.predict_proba(X.drop(['Team', 'games_played', 'season'], 1))[:,1]
predictions['pred_rf'] = RF.predict_proba(X.drop(['Team', 'games_played', 'season'], 1))[:,1]
predictions['pred_gbm'] = GBM.predict_proba(X.drop(['Team', 'games_played', 'season'], 1))[:,1]
predictions['pred_lgb'] = LGB.predict_proba(X.drop(['Team', 'games_played', 'season'], 1))[:,1]
predictions['pred_catboost'] = CATBOOST.predict_proba(X.drop(['Team', 'games_played', 'season'], 1))[:,1]
predictions['pred_mean'] = (predictions['pred_logit'] + predictions['pred_rf'] + predictions['pred_gbm'] + predictions['pred_lgb'] + predictions['pred_catboost']) / 5
predictions['pred_opt_combo'] = (predictions['pred_logit'] + 7*predictions['pred_catboost']) / 8

predictions.sort_values('pred_opt_combo', ascending = False, inplace = True)
predictions.head(3)

,Team,season,week_marker,pred_logit,pred_rf,pred_gbm,pred_lgb,pred_catboost,pred_mean,pred_opt_combo
0,Kansas City Chiefs,2022,week_11,0.430743,0.333554,0.221133,0.322177,0.416739,0.344869,0.418490
1,Buffalo Bills,2022,week_11,0.548144,0.163632,0.099688,0.216142,0.370342,0.279590,0.392567
7,San Francisco 49ers,2022,week_11,0.343739,0.198990,0.122062,0.205453,0.357053,0.245459,0.355388


In [10]:
predictions.to_csv('weekly_runs_2022/' + week_name + '.csv', index = False)

### Check weekly run

In [11]:
predictions

,Team,season,week_marker,pred_logit,pred_rf,pred_gbm,pred_lgb,pred_catboost,pred_mean,pred_opt_combo
0,Kansas City Chiefs,2022,week_11,0.430743,0.333554,0.221133,0.322177,0.416739,0.344869,0.418490
1,Buffalo Bills,2022,week_11,0.548144,0.163632,0.099688,0.216142,0.370342,0.279590,0.392567
7,San Francisco 49ers,2022,week_11,0.343739,0.198990,0.122062,0.205453,0.357053,0.245459,0.355388
4,Philadelphia Eagles,2022,week_11,0.292566,0.261418,0.158623,0.256253,0.357324,0.265237,0.349229
2,Miami Dolphins,2022,week_11,0.074864,0.280978,0.120186,0.276535,0.374377,0.225388,0.336937
10,Dallas Cowboys,2022,week_11,0.216692,0.088931,0.074256,0.197687,0.347889,0.185091,0.331490
5,Cincinnati Bengals,2022,week_11,0.043475,0.119375,0.067294,0.228347,0.362471,0.164192,0.322597
11,Seattle Seahawks,2022,week_11,0.012243,0.054839,0.058485,0.195959,0.348906,0.134087,0.306824
16,Tampa Bay Buccaneers,2022,week_11,0.029480,0.072168,0.071544,0.205928,0.344500,0.144724,0.305122
22,New England Patriots,2022,week_11,0.050456,0.051460,0.075758,0.222721,0.340083,0.148096,0.303880
